# Analysis of mobile apps on Google Play and the App Store

For this project, I'll pretend I am working as data analysts for a company that builds Android and iOS mobile apps. We make our apps available on Google Play and the App Store. We will analyze the data of currents mobile apps. Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

# Opening and exploring data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. Collecting data for over 4 million apps requires a significant amount of time and money, so I'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, I first try to see if I can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our goals: AppleStore.csv and googleplaystore.csv. I'll start by opening and exploring these two data sets.

In [1]:
opened_file_as = open('AppleStore.csv')
opened_file_gp = open('googleplaystore.csv')
from csv import reader
read_file_as = reader(opened_file_as)
read_file_gp = reader(opened_file_gp)
as_dataset = list(read_file_as)
gp_dataset = list(read_file_gp)

### We write a function to explore the two data sets. ###
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
explore_data(as_dataset, 1, 5, rows_and_columns = True)
explore_data(gp_dataset, 1, 5, rows_and_columns = True)
print(as_dataset[0])
print('\n')
print(gp_dataset[0])

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7198
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live 

# Deleting wrong data

The Google Play data set has a dedicated discussion section, and we can see
that one of the discussions describes an error for the row 10473. Now let's print row 10473 to check detail information.

In [2]:
print(gp_dataset[10473])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Rating for app 'Life Made WI-Fi Touchscreen Photo Frame' is 19. But for Google Play app, the value of rating is no more than 5. So I need to delete this row.

In [3]:
del gp_dataset[10473]

# Removing duplicate entries
If we explore the Google Play data set long enough or look at the discussion section, we'll notice some apps have duplicate entries. For instance, let's check the number of entries for 'Instagram'.

In [4]:
for row in gp_dataset:
    name = row[0]
    if name == 'Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Let's count the number of duplicates in Google Play dataset.

In [5]:
unique = []
duplicates = []
for row in gp_dataset:
    name = row[0]
    if name in unique:
        duplicates.append(name)
    else:
        unique.append(name)
print("Number of duplicates: ", len(duplicates))

Number of duplicates:  1181


So there are 1181 duplicates in Google Play dataset. To clean up the data set, I need to remove the duplicates. 
Based on the row of 'Instagram', the main difference among the duplicates is the fourth column, which indicates the reviews. I will go with the entry with the most reviews. Because most reviews mean that it is the most popular one.

In [6]:
### Create a frequency table of reviews of the apps in Google Play Store. ###
reviews_max = {}
for name in duplicates:
    for row in gp_dataset:
        if row[0] == name:
            review = float(row[3])
            if name not in reviews_max:
                reviews_max[name] = review
            elif name in reviews_max and review > reviews_max[name]:
                reviews_max[name] = review
### Clean the data according to the frequency table of review. ###
app_added = [] 
gp_dataset_cleaned = []
gp_dataset_cleaned.append(gp_dataset[0])
for row in gp_dataset[1:]:
    n_name = row[0]
    n_review = float(row[3])
    if n_name not in duplicates:
        gp_dataset_cleaned.append(row)
        app_added.append(n_name)
    elif n_name not in app_added and n_review == reviews_max[n_name]:
        gp_dataset_cleaned.append(row)
        app_added.append(n_name)

print(len(app_added))
print(len(gp_dataset_cleaned))
print(len(gp_dataset))
                
            

9659
9660
10841


Based on the above results there are 9659 (=10841 - 1 - 1181) unique apps in Google Play Store. This matches the previous result of duplicate apps.

# Removing non-English apps
Now we will analyze the content of each app. Remember that we are only interested in English app. So we need to find those non-English app. Next, we will write a function to check any character in the string that doesn't belong to the set of common English characters.

In [7]:
def English_ornot(string):
    for c in string:
        if ord(c)>127:
            return False
    return True   
    

Let's test our function using some app names.

In [8]:
English_ornot('Instagram')

True

In [9]:
English_ornot('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [10]:
English_ornot('Docs To Go™ Free Office Suite')

False

In [11]:
English_ornot('Instachat 😜')

False

From the examples above, we can see that we will lose some useful data with our current function. Let's edit the function we wrote before to filter out the non-English apps.

We will change the previous function we created before. It the input string has more than three characters that fall outside the ASCII range (0-127), then the function should return False (identify the string as non-English), otherwise it should return True.

In [12]:
# Rewrite the function to filter out non-English apps more precisely.###
def English_ornot(string):
    count = 0
    for c in string:
        if ord(c)>127:
            count += 1
            if count > 3:
                return False
    return True 

In [13]:
English_ornot('Docs To Go™ Free Office Suite')

True

In [14]:
English_ornot('Instachat 😜')

True

In [15]:
English_ornot('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

Use the new function to filter out non-English apps from both data sets. If an app name is identified as English, append the whole row to a separate list.

In [16]:
as_dataset_Eng = []
gp_dataset_Eng = []
### Filter out non-English apps from both datasets with the new function. ###
for row in as_dataset:
    name = row[0]
    if English_ornot(name):
        as_dataset_Eng.append(row)
        
for row in gp_dataset_cleaned:
    name = row[0]
    if English_ornot(name):
        gp_dataset_Eng.append(row)
        
print('Number of English apps in Apple Store: ', len(as_dataset_Eng))
print('Number of English apps in Google Play Store: ', len(gp_dataset_Eng))
        

Number of English apps in Apple Store:  7198
Number of English apps in Google Play Store:  9615


# Isolating free apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

In [17]:
free_Eng_AS_dataset = []
free_Eng_AS_dataset.append(as_dataset_Eng[0])
free_Eng_GP_dataset = []
free_Eng_GP_dataset.append(gp_dataset_Eng[0])
for row in as_dataset_Eng[1:]:
    price = row[4]
    if price == '0.0':
        free_Eng_AS_dataset.append(row)
        
for row in gp_dataset_Eng[1:]:
    price = row[7]
    if price == '0':
        free_Eng_GP_dataset.append(row)
        
print('Number of free English apps in Apple Store: ', len(free_Eng_AS_dataset))
print('Number of free English apps in Google Play Store: ', len(free_Eng_GP_dataset))


Number of free English apps in Apple Store:  4057
Number of free English apps in Google Play Store:  8865


# Finding out the most popular apps

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

In [18]:
print(free_Eng_AS_dataset[0])
print(free_Eng_GP_dataset[0])

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [19]:
# Create frequency tables of 'prime-genre' in App Store dataset and 'Genres'
# in Google Play Store dataset. 
 
freeEng_AS_frequen = {}
freeEng_GP_frequen = {}
for row in free_Eng_AS_dataset[1:]:
    genrename = row[11]
    if genrename not in freeEng_AS_frequen:
        freeEng_AS_frequen[genrename] = 1
    else:
        freeEng_AS_frequen[genrename] += 1
        
for row in free_Eng_GP_dataset[1:]:
    genrename = row[9]
    if genrename not in freeEng_GP_frequen:
        freeEng_GP_frequen[genrename] = 1
    else:
        freeEng_GP_frequen[genrename] += 1
        
print(freeEng_AS_frequen)
print(freeEng_GP_frequen)

{'Weather': 31, 'Travel': 56, 'News': 58, 'Reference': 20, 'Catalogs': 9, 'Medical': 8, 'Entertainment': 334, 'Health & Fitness': 76, 'Games': 2257, 'Sports': 79, 'Finance': 84, 'Productivity': 62, 'Education': 132, 'Business': 20, 'Music': 67, 'Lifestyle': 94, 'Shopping': 121, 'Book': 66, 'Utilities': 109, 'Photo & Video': 167, 'Social Networking': 143, 'Food & Drink': 43, 'Navigation': 20}
{'Weather': 71, 'Arcade': 164, 'Role Playing;Pretend Play': 4, 'Education': 474, 'Sports;Action & Adventure': 2, 'Art & Design;Creativity': 6, 'Photography': 261, 'Parenting;Brain Games': 1, 'Educational;Education': 35, 'Music & Audio;Music & Video': 1, 'Racing;Pretend Play': 1, 'Travel & Local': 206, 'Sports': 307, 'Health & Fitness;Education': 1, 'Entertainment;Education': 1, 'Productivity': 345, 'Entertainment;Creativity': 3, 'Books & Reference;Education': 1, 'Racing': 88, 'Board;Brain Games': 7, 'Education;Brain Games': 3, 'Personalization': 294, 'Adventure': 60, 'Racing;Action & Adventure': 15

Let's find out the most common apps in both datasets.

In [20]:
freeEng_AS_frequen_rev = {}
freeEng_GP_frequen_rev = {}
for key, value in freeEng_AS_frequen.items():
    freeEng_AS_frequen_rev[value] = key
    
for key, value in freeEng_GP_frequen.items():
    freeEng_GP_frequen_rev[value] = key 

for i in sorted(freeEng_AS_frequen_rev, reverse = True):
    print('The most common genres in Apple Store is: ', freeEng_AS_frequen_rev[i], ' : ', i)
    break
for i in sorted(freeEng_GP_frequen_rev, reverse = True):
    print('The most common genres in Google Play Store is: ', freeEng_GP_frequen_rev[i], ' : ', i)
    break
    

The most common genres in Apple Store is:  Games  :  2257
The most common genres in Google Play Store is:  Tools  :  749


There is another way to do the same ting. Let's now create a function for generating frequency tables, and use it in combination with the display_table() function.

In [21]:
def freq_table(dataset, index):
    fre_tab = {}
    for row in dataset:
        name = row[index]
        if name not in fre_tab:
            fre_tab[name] = 1
        else:
            fre_tab[name] += 1
    for key, value in fre_tab.items():
        fre_tab[key]=round((value/len(dataset)) * 100, 2)
    return fre_tab

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [22]:
display_table(free_Eng_AS_dataset, 11)
print('\n')
display_table(free_Eng_GP_dataset, 9)
print('\n')
display_table(free_Eng_GP_dataset, 1)

Games : 55.63
Entertainment : 8.23
Photo & Video : 4.12
Social Networking : 3.52
Education : 3.25
Shopping : 2.98
Utilities : 2.69
Lifestyle : 2.32
Finance : 2.07
Sports : 1.95
Health & Fitness : 1.87
Music : 1.65
Book : 1.63
Productivity : 1.53
News : 1.43
Travel : 1.38
Food & Drink : 1.06
Weather : 0.76
Reference : 0.49
Navigation : 0.49
Business : 0.49
Catalogs : 0.22
Medical : 0.2
prime_genre : 0.02


Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.24
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.92
Strategy : 0.9

The frequency tables we analyzed on the previous screen showed us that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and fun apps. Now, we'd like to get an idea about the kind of apps with the most users.

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app. We'll use a nested loop to compute the averages we mentioned above.

In [23]:
def freq_table_number(dataset, index):
    fre_tab = {}
    for row in dataset:
        name = row[index]
        if name not in fre_tab:
            fre_tab[name] = 1
        else:
            fre_tab[name] += 1
    return fre_tab

def display_table(table):
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        
frequency_table_num_AS=freq_table_number(free_Eng_AS_dataset[1:], 11)
avgrating_table = {}
for genre in frequency_table_num_AS:
    total = 0
    for row in free_Eng_AS_dataset:
        if row[11] == genre:
            total += int(row[5])
    avgrating_table[genre] = round(total/frequency_table_num_AS[genre])
    
display_table(avgrating_table)

Reference : 67448
Music : 56482
Social Networking : 53078
Weather : 47221
Photo & Video : 27250
Navigation : 25972
Travel : 20216
Food & Drink : 20179
Sports : 20129
Health & Fitness : 19952
Productivity : 19054
Games : 18925
Shopping : 18747
News : 15893
Utilities : 14010
Finance : 13522
Entertainment : 10823
Lifestyle : 8978
Book : 8498
Business : 6368
Education : 6266
Catalogs : 1780
Medical : 460


Based on the result, apps for 'Reference' are most popular in App Store. Let's look at some examples.

In [24]:
i = 0
for row in free_Eng_AS_dataset:
    if row[11] == 'Reference':
        print(row)
        i += 1
        if i > 10:
            break

['282935706', 'Bible', '92774400', 'USD', '0.0', '985920', '5320', '4.5', '5.0', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']
['308750436', 'Dictionary.com Dictionary & Thesaurus', '111275008', 'USD', '0.0', '200047', '177', '4.0', '4.0', '7.1.3', '4+', 'Reference', '37', '0', '1', '1']
['364740856', 'Dictionary.com Dictionary & Thesaurus for iPad', '165748736', 'USD', '0.0', '54175', '10176', '4.5', '4.5', '4.0', '4+', 'Reference', '24', '5', '9', '1']
['414706506', 'Google Translate', '65281024', 'USD', '0.0', '26786', '27', '3.5', '4.5', '5.10.0', '4+', 'Reference', '37', '5', '59', '1']
['388389451', 'Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran', '100551680', 'USD', '0.0', '18418', '706', '4.5', '5.0', '9.2.1', '4+', 'Reference', '37', '5', '16', '1']
['1130829481', 'New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition', '52959232', 'USD', '0.0', '17588', '17588', '4.5', '4.5', '1.0', '4+', 'Reference', '38', '3', '2', '1']
['399452287', 'Merriam-Webst

Now let's take a look at some examples of apps from 'Social Networking'. 

In [25]:
i = 0
for row in free_Eng_AS_dataset:
    if row[11] == 'Social Networking':
        print(row)
        i += 1
        if i > 10:
            break

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']
['429047995', 'Pinterest', '74778624', 'USD', '0.0', '1061624', '1814', '4.5', '4.0', '6.26', '12+', 'Social Networking', '37', '5', '27', '1']
['304878510', 'Skype for iPhone', '133238784', 'USD', '0.0', '373519', '127', '3.5', '4.0', '6.35.1', '4+', 'Social Networking', '37', '0', '32', '1']
['454638411', 'Messenger', '275729408', 'USD', '0.0', '351466', '892', '3.0', '3.0', '119.0', '4+', 'Social Networking', '37', '1', '33', '1']
['305343404', 'Tumblr', '151573504', 'USD', '0.0', '334293', '919', '4.0', '4.0', '8.6', '17+', 'Social Networking', '37', '5', '16', '1']
['310633997', 'WhatsApp Messenger', '135044096', 'USD', '0.0', '287589', '73088', '4.5', '4.5', '2.17.22', '4+', 'Social Networking', '12', '0', '35', '1']
['357218860', 'Kik', '151864320', 'USD', '0.0', '260965', '228', '4.0', '3.0', '11.21.0', '12+', 'Social Networking', '37', '

Now let's calculate the average number of installs per app genre for the Google Play data set. We'll need to use a nested loop. The install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.). For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to find out which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on. To perform computations, however, we'll need to convert each install number from string to float. This means we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error.

In [26]:
frequency_table_num_GP=freq_table_number(free_Eng_GP_dataset[1:], 9)
avgrating_table_GP = {}
for genre in frequency_table_num_GP:
    total = 0
    for row in free_Eng_GP_dataset:
        if row[9] == genre:
            instals = row[5]
            instals = instals.replace('+', '')
            instals = instals.replace(',', '')
            total += int(instals)
    avgrating_table_GP[genre] = round(total/frequency_table_num_GP[genre],2)

display_table(avgrating_table_GP)

Communication : 38456119.17
Adventure;Action & Adventure : 35333333.33
Video Players & Editors : 24947335.8
Social : 23253652.13
Arcade : 22888365.49
Casual : 19569221.6
Puzzle;Action & Adventure : 18366666.67
Photography : 17840110.4
Educational;Action & Adventure : 17016666.67
Productivity : 16787331.34
Racing : 15910645.68
Travel & Local : 14051476.15
Casual;Action & Adventure : 12916666.67
Action : 12603588.87
Strategy : 11199902.53
Tools : 10802461.25
Tools;Education : 10000000.0
Role Playing;Brain Games : 10000000.0
Lifestyle;Pretend Play : 10000000.0
Casual;Music & Video : 10000000.0
Card;Action & Adventure : 10000000.0
Adventure;Education : 10000000.0
News & Magazines : 9549178.47
Music : 9445583.33
Educational;Pretend Play : 9375000.0
Puzzle;Brain Games : 9280666.67
Word : 9094458.7
Racing;Action & Adventure : 8816666.67
Books & Reference : 8767811.89
Puzzle : 8302861.91
Video Players & Editors;Music & Video : 7500000.0
Shopping : 7036877.31
Role Playing;Action & Adventure : 7

Based on the result, apps for 'Communication' are most popular in Google Play Store. Let's look at some examples.

In [27]:
i = 0
for row in free_Eng_GP_dataset:
    if row[9] == 'Communication':
        print(row)
        i += 1
        if i > 10:
            break

['WhatsApp Messenger', 'COMMUNICATION', '4.4', '69119316', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Everyone', 'Communication', 'August 3, 2018', 'Varies with device', 'Varies with device']
['Messenger for SMS', 'COMMUNICATION', '4.3', '125257', '17M', '10,000,000+', 'Free', '0', 'Teen', 'Communication', 'June 6, 2018', '1.8.9', '4.1 and up']
['My Tele2', 'COMMUNICATION', '4.3', '158679', '8.8M', '5,000,000+', 'Free', '0', 'Everyone', 'Communication', 'August 3, 2018', '2.4.1', '4.4 and up']
['imo beta free calls and text', 'COMMUNICATION', '4.3', '659395', '11M', '100,000,000+', 'Free', '0', 'Everyone', 'Communication', 'June 7, 2018', '9.8.000000010492', '4.0 and up']
['Contacts', 'COMMUNICATION', '4.3', '66602', 'Varies with device', '50,000,000+', 'Free', '0', 'Everyone', 'Communication', 'June 26, 2018', '2.8.4.201036949', '5.0 and up']
['Call Free – Free Call', 'COMMUNICATION', '4.3', '30209', '15M', '5,000,000+', 'Free', '0', 'Everyone', 'Communication', 'July 28, 2

It came up that apps for social communication are the most popular. We can try to launch new social app that is more user-friendly.

# Conclusions

In this project, we analyzed data about the App Store and Google Play Store apps with the goal of recommending an app profile that can be profitable for both markets. 

It turns out that social networking and communication apps are most common among the users. We can try to launch our new social app that is more user-friendly, liking being able to share larger vedio, to attract more users.